## Installs

In [ ]:
!sudo apt-get install fluidsynth

!pip install -qq gdown pyfluidsynth pretty_midi torch numpy pandas midi-player multiprocess wandb


## Imports

In [ ]:
from IPython.display import display, HTML
import collections
import datetime
import fluidsynth
import glob
import pandas as pd
import pretty_midi

import torch
from torch import nn, einsum
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from typing import Optional
from glob import glob
from pathlib import Path
import multiprocess as mp   # multiprocess is a Jupyter-compatible fork of multiprocessing
from functools import partial

from tqdm import tqdm  # note: Quarto blog won't print output from tqdm.notebook
#from tqdm.contrib.concurrent import process_map  # process_map throws errors on Mac :'-( 

import random
import numpy as np 
import pprint

from collections import defaultdict 
import matplotlib.pyplot as plt

from midi_player import MIDIPlayer
from midi_player.stylers import general, dark # I like dark mode

from dataclasses import dataclass

import wandb 


## MIDI Dataset Stuff

In [ ]:

# Get the data
data_source = 'jsb_chorales_midi'
# TODO: add options for MAESTRO, others

data_dir = Path('midi_data') # generic name for whatever midi we might want
REST_PITCH = 127  # special code  used to denote rests

!gdown -O {data_source}.tgz '1MdJiNEgtkvCx9tnyQWcnEE5GPMY6ADUb'
!tar zxf {data_source}.tgz; rm -rf midi_data; ln -s {data_source} midi_data ;

# get the list of MIDI filenames
filenames = sorted(glob(str(data_dir/'**/*.mid*'),recursive=True))
print('Number of files:', len(filenames))



midi_file = filenames[0]
MIDIPlayer(midi_file, 360, styler=dark, title=f"midi_file = {midi_file}")


def quantize_times(times, 
                   res_ms=8, # resolution in milliseconds. 8ms was deemed sufficient for MAESTRO.
                   clamp_range_s=[0.0,4.0]):
    "reasonably coarse time quantization is really helpful for facilitating learning temporal behavior"
    quant_step = res_ms / 1000.0
    q_times = torch.round(times / quant_step) * quant_step
    if clamp_range_s is not None: 
        q_times = torch.clamp(q_times, clamp_range_s[0], clamp_range_s[1])
    return q_times


def midi_file_to_tensor(filenames, i=None, keep_start_end=False, rest_pitches=[REST_PITCH]):  # filenames could just be a single filename
    midi_file = filenames if i is None or type(filenames)==str  else filenames[i]
    "reads a single midi file and converts it to tensor with elements (pitch, step, duration)"
    pm = pretty_midi.PrettyMIDI(midi_file) # read in the whole file. this can be very slow for long MIDI files (e.g. in MAESTRO)

    # Sort the notes first by start time (and then by pitch if two notes start at the same time)
    sorted_notes = sorted(pm.instruments[0].notes, key=lambda note: (note.start, note.pitch))
    
    prev_start = sorted_notes[0].start
    notes = []
    for i, note in enumerate(sorted_notes):
        new_note =  torch.empty( (3 + 2*keep_start_end) , dtype=torch.float32)
        if int(note.pitch) in rest_pitches: continue  # we can actually delete the rests! 
        new_note[0] = note.pitch
        new_note[1] = note.start - prev_start  # step, i.e. time since start of previous note
        new_note[2] = note.end - note.start    # duration
        if keep_start_end:                     # might find it useful be able to keep these for easier analysis later
            new_note[3] = note.start
            new_note[4] = note.end
        prev_start = note.start
        notes.append(new_note)

    notes = torch.vstack(notes)
    notes[:,1:] = quantize_times(notes[:,1:])
    return notes

notes_tensor = midi_file_to_tensor(midi_file)



def files_to_tensor_list(filenames, keep_start_end=False, serial=True):
    "Reads MIDI files in parallel so should be reasonably fast. JSB Chorales are no prob but for MAESTRO you want this"
    # tensor_list = process_map(midi_file_to_tensor, filenames, max_workers=mp.cpu_count(), chunksize=1) # Doesn't work on Mac
    tensor_list = []
    max_ = len(filenames)
    if serial:
        for i, filename in enumerate(tqdm(filenames)):
            tensor_list.append(midi_file_to_tensor(filename,  keep_start_end=keep_start_end))
    else:
        with mp.Pool(processes=mp.cpu_count()) as p:
            with tqdm(total=max_) as pbar:
                for r in p.imap_unordered(partial(midi_file_to_tensor, filenames, keep_start_end=keep_start_end), range(0, max_)):
                    tensor_list.append(r)
                    pbar.update()
    return  tensor_list




notes_tensor_list_filename = f'{data_source}_tensor_list.pt'         # we'll save the result to a file for easier re-reading next time
read_all_midi_files = True # not os.path.exists(notes_tensor_list_filename) # check to see if it already exists

if read_all_midi_files:
    notes_tensor_list = files_to_tensor_list(filenames, serial=True)
    torch.save(notes_tensor_list, notes_tensor_list_filename) # save for next time
else:
    notes_tensor_list = torch.load(notes_tensor_list_filename) # just read from the last time we made one
    
print(f"\nlen(notes_tensor_list) = {len(notes_tensor_list)}")

notes_tl = notes_tensor_list
all_notes = torch.vstack(notes_tl).type(torch.float32)
print("all_notes.shape = ",all_notes.shape)



def make_codebooks(all_notes, verbose=False):
    codebooks = []
    n_codebooks = all_notes.shape[-1] # should be 3
    for i in range(n_codebooks): 
        if i==0:  # i=0 means pitch
            cb_vals = torch.arange(128)  # just use all possible pitches 
        else:     # i!=0 means timing
            cb_vals = all_notes[:,i].unique().sort()[0] 
        if verbose: print(f"\n---\ncb {i}: cb_vals = {cb_vals}")
        codebooks.append({'encode':{np.round(k.item(),3): int(v) for v, k in enumerate(cb_vals)}, # codebooks go both ways
                          'decode':{int(v): k for v, k in enumerate(cb_vals)}})
        if verbose: print(f" cb {i}: cb keys = {codebooks[-1]['encode'].keys()}")
    return codebooks

all_notes[:,1:] = quantize_times(all_notes[:,1:])
codebooks = make_codebooks(all_notes)

vocab_sizes = [len(cb['encode']) for cb in codebooks]
display(HTML(f"vocab_sizes = {vocab_sizes}"))




def remap_vals(seq, encdec_str, dtype=torch.long):
    out = torch.zeros_like(seq, dtype=dtype)
    for cb in range(seq.shape[-1]): 
        dict_map = codebooks[cb][encdec_str]
        default_map = max(codebooks[cb][encdec_str].values())
        out[:,cb] = torch.tensor([dict_map.get(np.round(x.item(),3),default_map) for x in seq[:,cb]], dtype=dtype) # that np.round(,3) is good for avoiding KeyErrors
        #out[:,cb] = torch.tensor([dict_map[np.round(x.item(),3)] for x in seq[:,cb]], dtype=dtype)
    return out 

# We'll be calling these 'encode' and 'decode functions', which really just pull up the applicable part of the codebooks
encode = lambda s: remap_vals(s, 'encode')
decode = lambda s: remap_vals(s, 'decode', dtype=all_notes.dtype)


# And let's do a little test
midi_seq = all_notes[0:6].clone()
print("Before encoding, midi_seq =\n",midi_seq)
token_list = encode(midi_seq)
print("After encoding, token_list =\n",token_list)
return_seq = decode(token_list)
print("After decoding, return_seq =\n",return_seq)
assert torch.equal(midi_seq, return_seq), f"Oops. midi_seq={midi_seq}, but return_seq={return_seq}. Should be the same"


midi_seq[-1,-1] = 100.0  # give the last time a huge value to check that our mapper won't crash
token_list = encode(midi_seq) # if it doesn't crash, we're good
assert token_list[-1,-1] == list(codebooks[-1]['encode'].values())[-1], "Big value should have gotten the last spot in the last codebook"

print("Checks pass! :-)")


def get_startend(notes:torch.Tensor) -> torch.Tensor:
    "integrates (step,duration) timing pairs to recover (start,end) info. concats them as new columns"
    newnotes = torch.zeros((len(notes), 5), dtype=notes.dtype, device=notes.device)
    newnotes[:,:3] = notes[:,:3]
    prev_start = 0.0
    for i, note in enumerate(notes): 
        step, dur = note[1], note[2]
        start = step  + prev_start 
        end   = start + dur
        newnotes[i,3], newnotes[i,4] = start, end
        prev_start = start   
    return newnotes
    

def notes_to_midi(notes:torch.Tensor, 
                  time_rescale=None, 
                  out_file: str = '',
                  instrument_name: str = 'Acoustic Grand Piano',
                  velocity: int = 64,  # default loudness for all notes
                 ) -> pretty_midi.PrettyMIDI:
    notes = notes.clone() # just to avoid weird overwrites of memory addresses
    if notes.min() < 0.0:
      print("WARNING: You have negative pitches, steps or durations. Setting them to zero")
      notes = notes * (notes >= 0)
    if time_rescale is not None: # just added this because sometime I want to slow/speed up
        notes[:,1:] = notes[:,1:] *time_rescale
        
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(
        program=pretty_midi.instrument_name_to_program(
            instrument_name))
    
    if notes.shape[-1] < 5: notes = get_startend(notes)
        
    notes = notes.cpu().numpy()
    prev_start = 0.0
    for note in notes: 
        pitch, start, end = int(note[0]), note[3], note[4] 
        midi_note = pretty_midi.Note( velocity=velocity, pitch=pitch, start=start, end=end, )
        instrument.notes.append(midi_note)
        prev_start = start   
        
    pm.instruments.append(instrument)
    if out_file: pm.write(out_file)
    return pm



def midiplayer(notes_tensor, height=400, time_rescale=None, midi_file="/tmp/tmp.mid", title='', styler=dark):
    "MIDIplayer that writes input tensor to temporary file"
    pm = notes_to_midi(notes_tensor, time_rescale=time_rescale, out_file=midi_file)
    return MIDIPlayer(midi_file, height, styler=dark, dl=True, title=title)

midiplayer(decode(encode(midi_file_to_tensor(filenames[0]))), title='Encode-Decode Test')


# tensor lists
train_filenames = [x for x in filenames if '/train' in x]
val_filenames   = [x for x in filenames if '/val'   in x]
test_filenames  = [x for x in filenames if '/test'  in x]

train_notes_tl = files_to_tensor_list(train_filenames, serial=True)
val_notes_tl   = files_to_tensor_list(val_filenames, serial=True)
test_notes_tl  = files_to_tensor_list(test_filenames, serial=True)

for name, tl in zip(['train','val','test'],[train_notes_tl, val_notes_tl, test_notes_tl]):
    stack = torch.vstack(tl)
    print(f"{len(tl)} songs in {name}, {stack.shape[0]} notes")


def augment_data(data, pitch_shift=12, debug=True, extra_augs=False):
    datanew = data.clone()                                     # avoid overwriting memory of data
    # pitch
    change = torch.randint(-pitch_shift, pitch_shift, (1,))    # how many semitones to change all the pitches
    datanew[ datanew[:,0] != REST_PITCH ] += torch.tensor((change, 0, 0))  # change the pitches
    if not extra_augs: return datanew   #only do pitches

    if torch.rand(1) < 0.2:                         # sometimes invert pitches? Probably not useful but anyway
        datanew[ datanew[:,0] != REST_PITCH ] *= torch.tensor((-1, 1, 1))
        datanew[ datanew[:,0] != REST_PITCH ] += torch.tensor((127, 0, 0))
    
    # time - if we sometimes increase each non-zero time-token by one, that should be ok, right? 
    if torch.rand(1) < 0.2: # do step
        datanew[ datanew[:,1] > 0]  += torch.tensor((0,1,0))
    if torch.rand(1) < 0.2: # do duration
        datanew[ datanew[:,2] > 0]  += torch.tensor((0,0,1))
        
    # extra 'safety' constraint: clamp to range of valid values (of tokens)
    for i, cb in enumerate(codebooks):
        datanew[:,i] = torch.clamp(datanew[:,i], 0, len(cb['encode'])-1)
    return datanew


# testing code: 
torch.manual_seed(3) # setting this just  to make sure something happens ;-) 
data = torch.tensor([[54,12,6],[61,0,40],[127,14,4],[86,0,12],[126,7,12]])
print("data.shape = ",data.shape)
print("original data = \n",data)
aug = augment_data(data)
print("augmented data = \n",aug) # 

assert not torch.equal(aug[:,0], data[:,0]), "Oops, nothing changed"
assert aug[2,0]==data[2,0], "Oops,  The 127 got changed"
print("Checks passed! :-) ")


class NotesDataset(Dataset):
    "simple custom dataset of sliding windows"
    def __init__(self, 
                 tensor_list, 
                 seq_length:int, 
                 tokenizer=encode,
                 codebooks=codebooks, 
                 aug_callback=augment_data,
                 len_mult=100,  # factor to 'fudge' the dataset length when it's inspected by DataLoaders
                 pad=True, # pad end with rests
                ):
        super().__init__()
        self.sl = seq_length
        self.len_mult = len_mult
        self.data_list = [tokenizer(t) for t in tensor_list] # encoded tokens are all we'll use
        if pad:
            rests = torch.tensor([REST_PITCH,1,1]).unsqueeze(0).tile((seq_length,1))
            self.data_list = [torch.cat((toks,rests), dim=0) for toks in self.data_list]
        self.aug_callback = aug_callback

    def __len__(self):
        """we're going to be grabbing random windows from the data, so just the len of the tensor 
        list will be too small for large batch sizes, hence we multiply by len_mult"""
        return len(self.data_list)*self.len_mult  # this will keep the DataLoader going longer

    def __getitem__(self, idx, shift=1) -> (torch.Tensor, torch.Tensor):
        "grabs a random 'window' from a random song, with an offset of `shift` tokens between inputs and targets"
        i_song = torch.randint(0, len(self.data_list), (1,)) # pick a song
        ix =  torch.randint(0, len(self.data_list[i_song]) - self.sl - 1, (1,))  # start of window within song
        data_block = self.data_list[i_song][ix:ix+self.sl+1]  # grab window plus an extra character
        if self.aug_callback is not None: 
            data_block = self.aug_callback(data_block)
        inputs, targets = data_block[:self.sl], data_block[shift:self.sl+shift]
        return inputs, targets
    

seq_length = 64
train_ds = NotesDataset(train_notes_tl, seq_length)
val_ds = NotesDataset(val_notes_tl, seq_length, aug_callback=None, len_mult=1000000) # no aug, neverending
# save test_ds for Evaluation section, later
len(train_ds), len(val_ds)





batch_size = 128 # We may change this further down, for now it's just a test
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, )

batch_x, batch_y = next(iter(train_dl))
print("batch_x.shape, batch_y.shape = ",batch_x.shape, batch_y.shape)



## Model Architecture Stuff

In [ ]:

@dataclass
class MusicBoxConfig:
    # model architecture details
    seq_length: int = 64
    batch_size: int = 128
    n_embd:     int = 128     # embedding dimension to use for tokens & positions
    n_heads:    int = 8       # number of attention heads
    n_blocks:   int = 4       # number of attention blocks
    dropout:  float = 0.1     # dropout value applied everywhere
    bias:      bool = False   # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

    # training details
    learning_rate: float = 0.001
    weight_decay:  float = 0.01   # 0.01 is pytorch default
    epochs:          int = 20

    # other handy bookkeeping
    vocab_sizes: tuple = tuple(vocab_sizes)
 
        
config = MusicBoxConfig()
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(config)


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, config):
        super().__init__()
        n_embd, block_size = config.n_embd, config.seq_length
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size, config):
        super().__init__()
        n_embd = config.n_embd

        self.heads = nn.ModuleList([Head(head_size, config) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

    
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd, config):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(config.dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, config):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, config)
        self.ffwd = FeedFoward(n_embd, config)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class Transformer(nn.Module):

    def __init__(self, config, debug=False):
        super().__init__()
        n_cb = len(config.vocab_sizes)
        self.block_size, n_embd, n_head, n_layer = config.seq_length, config.n_embd, config.n_heads, config.n_blocks 
        
        # seperate embeddings for pitch, step & dur part of notes
        self.token_embedding_tables = nn.ModuleList([nn.Embedding(vocab_sizes[cbi], n_embd) for cbi in range(n_cb)])

        self.position_embedding_table = nn.Embedding(self.block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, config) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_heads = nn.ModuleList([nn.Linear(n_embd, vocab_sizes[cbi]) for cbi in range(n_cb)]) # output token predictors
        self.debug=False

    def forward(self, idx, targets=None):
        # idx is array of input token indices in the current context
        B, T, CBS = idx.shape
        
        tok_emb = 0
        for cb in range(CBS): # just sum codebook reps
            tok_emb = tok_emb + self.token_embedding_tables[cb](idx[:,:,cb])
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,E)
        x = tok_emb + pos_emb # sum token embeddings & positional embeddings
        
        x = self.blocks(x) # Main computation loop! 
        x = self.ln_f(x)   # final layernorm
        logits_list = [head(x) for head in self.lm_heads]  # list of output projections over all codebook values

        if targets is None:  # need targets to compute loss
            loss = None
        else:
            lambdas = [0.5]*CBS   # relative "weights" to pitch, step, dur parts of loss
            loss = 0.0
            for cb in range(CBS):        # loop over codebooks  (for pitch, step & dur), summing loss
                logits = logits_list[cb]  
                B, T, V = logits.shape   # V = vocab size, i.e. codebook length
                targ = targets[:,:,cb]   # B, T 
                logits = logits.view(B*T, V)
                targ = targ.reshape(B*T)    # needs reshape & not view b/c of contiguous memory issues.
                loss = loss +  lambdas[cb] * F.cross_entropy(logits, targ)
        return logits_list, loss


    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0):
        # idx is (B, T, CBS) array of token indices in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]      # crop idx to the last block_size tokens
            logits_list, loss = self(idx_cond)   # get the predictions
            idx_next_list = []
            for cb in range(idx_cond.shape[-1]):
                # focus only on the last time step
                logits = logits_list[cb]  # B, T, V  where V = vocab/embedding size 
                logits = logits[:, -1, :] # get last time.  becomes (B, V)
                # apply softmax to get probabilities
                probs = F.softmax(logits/temperature, dim=-1) # (B, V)
                # sample from the distribution
                idx_next_list.append(torch.multinomial(probs, num_samples=1)) # (B, 1)
                
            idx_next = torch.tensor(idx_next_list).unsqueeze(0).unsqueeze(0).to(idx.device)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    
# test that, make sure we don't get any errors: 
model = Transformer(config)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters in the model') 





## Stuff Before Training

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("device is",device)


demo_prompt_idx = 0    # file index from which to pull the demo prompt
demo_target = val_notes_tl[demo_prompt_idx]
display(midiplayer(demo_target, title=f"Full demo 'target', {len(demo_target)} notes in length"))

demo_prompt_length = 16  # number of notes in demo prompt context
demo_max_new_tokens = min(150, len(demo_target))  # try to make the whole song, but don't go on too long
prompt = demo_target[:demo_prompt_length]
display(midiplayer(prompt, title=f"{demo_prompt_length}-note 'prompt' for demos"))
prompt_tokens = encode(prompt).unsqueeze(0).to(device)

#  set RNG seeds for reproducibility.
def set_seeds(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
        
set_seeds(0)

train_ds = NotesDataset(train_notes_tl, config.seq_length, len_mult=config.batch_size)
val_ds = NotesDataset(val_notes_tl, seq_length, aug_callback=None, len_mult=1000000) # no aug, neverending

train_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
val_dl   = DataLoader(val_ds,   batch_size=config.batch_size, shuffle=False)

model = Transformer(config)
#model = BigramLanguageModel()
model = model.to(device)
total_params_str = f'{sum(p.numel() for p in model.parameters())/1e6} M'
print(total_params_str,'parameters in the model')  

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

use_wandb = True  # Tip: leave this off at first, until you're sure everything's working!
if use_wandb:
    
    wandb.login()
    wandb.init(project="musicbox-jsb-tutorial", config=config)


# Training stuff

In [ ]:

val_every  = 1 # in steps, evaluate loss on val dataset
cp_every   = 60 # in epochs, checkpoint every
demo_every = 4 # in epochs, make a midi player demo

best_loss = 999

ema_weight, loss_ema, val_loss_ema = 0.95, None , None # exponential moving averages for loss reporting
step = 0 
losses = {}
epochs = config.epochs
for epoch in range(1,epochs+1):
    pbar = tqdm(total=len(train_dl), desc=f"Epoch {epoch}/{config.epochs}", dynamic_ncols=False, ncols=100) # progress bar, per epoch
    for bi, batch in enumerate(train_dl):
        step += 1
        xb, yb = batch[0].to(device), batch[1].to(device)

        logits, loss = model(xb, yb) # evaluate the loss
        loss_ema = (1.0-ema_weight)*loss.item() + ema_weight*loss_ema if loss_ema is not None else loss.item()
        losses['train'], losses['train_ema'] = loss.item(), loss_ema
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        # status / diagnostics:
        if (step % val_every == 0):
            with torch.no_grad():
                model.eval()
                xvb, yvb = next(iter(val_dl))
                val_logits, val_loss = model( xvb.to(device), yvb.to(device) ) 
                val_loss_ema =  (1.0-ema_weight)*val_loss.item() + ema_weight*val_loss_ema if val_loss_ema is not None else val_loss.item() 
                losses['val'], losses['val_ema'] = val_loss.item(), val_loss_ema
            model.train()
            
        wbl_dict = {'step':step, 'epoch':epoch} | losses   # dict for logging losses, midi examples, etc to wandb
        pbar.set_postfix( dict((k,losses[k]) for k in ['train', 'val','val_ema'])) # loss info for progress bar
        pbar.update(1)
        if use_wandb and wbl_dict != {}: wandb.log(wbl_dict)
            
    #--- end of epoch ---

    if losses['val_ema'] < best_loss: # Tracking best val_loss_ema for checkpointing purposes
        best_loss = losses['val_ema']
        pbar.set_postfix(dict( (k,losses[k]) for k in ['train', 'val']) | {'BEST val_ema':best_loss})
        if (epoch) % cp_every==0:   # occasionally save a checkpoint of best model/optimizer states
            cp_file = f"musicbox-jsb" #    -{step}" # let's leave out step to avoid filling disk
            print(f"Saving a checkpoint to {cp_file}")
            torch.save({ 'step': step, 'model_state_dict': model.state_dict(), 'loss': loss,
                         'optimizer_state_dict': optimizer.state_dict(),}, cp_file)
    pbar.refresh()
    pbar.close()
    
    if (epoch % demo_every == 0) or (epoch==epochs):  # demo of midi generation
        with torch.no_grad():
            model.eval()
            new_notes = decode( model.generate(prompt_tokens, max_new_tokens=demo_max_new_tokens, temperature=1)[0].cpu() )
            p2 = midiplayer(new_notes,title=f"Demo on val dataset, Epoch={epoch}")
            display(p2)
            if use_wandb: wandb.log( {'step':step, 'player':wandb.Html(p2.html)} )              
        model.train()


if use_wandb: wandb.finish()




## Evaluation Stuff

In [ ]:



file_ind = 1  
original = test_notes_tl[file_ind]
num_tokens = len(original)
display(midiplayer(original, title=f"Full Evaluation Target, {num_tokens} notes long"))


prompt_len = 21
prompt = original[:prompt_len] 
display(midiplayer(prompt, title=f"Evaluation Prompt, {prompt_len} notes long"))
prompt_tokens = encode(prompt).unsqueeze(0).to(device)

new_tokens = num_tokens - prompt_len

for temperature in [ 0.7, 0.85, 0.92, 1.0, 1.2, 1.5]:
    set_seeds(1337) # same temp for same seed will yield same output
    notes = decode( model.generate(prompt_tokens, max_new_tokens=new_tokens, temperature=temperature)[0].cpu() )
    display(midiplayer(notes, title=f"Temperature = {temperature}"))


test_ds = NotesDataset(test_notes_tl, config.seq_length, len_mult=config.batch_size, pad=False)
test_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=False)

total, batches = 0, 0 
with torch.no_grad():
    model.eval()
    for i, batch in enumerate(tqdm(test_dl, ncols=100)): 
        batches +=1 
        xb, yb = [q.to(device) for q in batch]
        logits, loss = model(xb, yb)
        total = total + loss.cpu().item()
perplexity = np.exp(total/batches) 
print(f"(Average) Perplexity = {perplexity:5.3f}")



all_test_notes = torch.vstack(test_notes_tl).type(torch.float32)
all_test_notes = all_test_notes[:len(all_test_notes)//4]  # to save time, let's not do stats on ALL-all of them, just a quarter

# generate a big batch of fake data. 
# WARNING: this is super slow because our generator only works for batches of one, oops
prompt_len, max_new_tokens = 12, 54          # I just made these numbers up. not great science yet, sorry.
new_notes_list = []
print("Generating new notes...") 
for i, notes in enumerate(tqdm(test_notes_tl, ncols=100)): 
    ptoks = encode(notes[:prompt_len]).unsqueeze(0).to(device)
    # generate more notes, chop off prompt before adding to list of all generated notes
    new_notes_list.append( decode( model.generate(ptoks, max_new_tokens=max_new_tokens)[0].cpu()[prompt_len:] ) )
all_new_notes = torch.vstack(new_notes_list).type(torch.float32)

print(f"# of test notes = {len(all_test_notes)},  # of generated notes = {len(all_new_notes)} --- Close enough!")


fig, ax = plt.subplots(1, 3, figsize=(15, 5))
names = ['pitch', 'step', 'duration']
for i in range(3):
    bins = 32 if i !=0 else 0.5 + 2*np.arange(128//2) 
    ax[i].hist(all_test_notes[:,i].numpy(), bins=bins, label='test data', alpha=0.6)
    ax[i].hist(all_new_notes[:,i].numpy(),  bins=bins, label='generated', alpha=0.6)
    ax[i].set_title(names[i])
    if i==2: ax[i].legend()
    ax[i].set_xlabel('Value'+f'{" (s)" if i>0 else ""}')
    if i==0: ax[i].set_ylabel('Count')
plt.show()